In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import numpy as np
import cv2
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
cap = cv2.VideoCapture('./train/trainGray_Clipped.m4v')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
pixelCount = frameHeight*frameWidth
print(frameCount, frameHeight, frameWidth, pixelCount)

bufTemp = np.empty((frameCount, frameHeight * frameWidth), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, x = np.array(cap.read())
    x = x[:,:,0]
    x.shape = frameHeight*frameWidth
    bufTemp[fc] = x
    fc += 1

cap.release()

# x = bufTemp[0, 0:frameHeight*frameWidth]
# x.shape = (frameHeight, frameWidth)

20400 170 320 54400


In [ ]:
x = np.empty((11, 10), np.dtype('float32'))
for i in range(11):
    x[i] = np.arange(i+5,i+15)
    print(x[i])
    mean = np.mean(x[i,:])
    x[i,:] = x[i,:]-mean
    print(x[i,:])
    print(mean,'\n')

In [ ]:
print(x)
x[:,0] = np.array([1,2,3,4,5,6,7,8,9,10,11],np.dtype('float32'))
print(x)
print(x.shape[0],x.shape[1])

In [3]:
x = np.empty((frameCount-1, (frameHeight*frameWidth)+1), np.dtype('float32'))
# print(x.shape)
# print(x[0].shape)
# x[0,0] = 1
# print(type(x[0,0]))
# print(x[0,0])
for j in range(1,frameCount):
    i = j-1
    firstFrame = np.array(bufTemp[i], dtype='int16')
    secondFrame = np.array(bufTemp[j], dtype='int16')
    x[i,0] = 1
    x[i,1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
    x[i,:] = np.absolute(x[i,:])
#     mean = np.mean(x[i,:])
#     x[i,:] = x[i,:] - mean
#     x[i,:] = x[i,:]/np.std(x[i,:])

for j in range(1):
    mean = np.mean(x[:,j])
    x[:,j] = x[:,j] - mean
    if np.sum(x[:,j]) != 0:
        x[:,j] = x[:,j]/np.std(x[:,j])

# h = np.std(x[:,0])
# print(h)
# print(x[:,0])
# print()
# x[:,0] = x[:,0]/h
# print(x[:,0])

In [4]:
print(x[0])

[  0.   0.   0.   1.   1.   0.   0.   1.   1.   0.   0.   2.   2.   0.
   0.   2.   2.   4.   4.   4.   4.   4.   4.   4.   4.   5.   5.   5.
   5.   3.   3.   3.   3.   2.   2.   2.   2.   2.   2.   2.   2.   6.
   6.   6.   6.   5.   5.   5.   5.   2.   4.   4.   4.   4.   4.   4.
   4.   4.   8.   8.   8.   7.   6.   6.   6.   7.   8.   8.   8.   8.
   9.   9.   9.   7.   6.   6.   6.   5.   6.   6.   6.   4.   6.   6.
   6.   6.   6.   6.   6.   9.   9.   9.   9.   9.   9.   9.   9.   7.
   7.   7.   7.   7.   7.   7.   7.   7.   7.   7.   7.   7.   7.   7.
   7.   7.   7.   7.   7.   7.   7.   7.   7.   9.   9.   9.   9.   9.
   9.   9.   9.   9.   9.   9.   9.   9.   9.   9.   9.   8.   9.   9.
   9.   9.   9.   9.   9.   9.   9.   9.   9.   9.   8.   8.   8.   9.
   9.   9.   9.   9.   8.   8.   8.   3.   1.   1.   1.   1.   1.   1.
   1.   3.   2.   2.   2.   2.   2.   2.   2.   2.   2.   2.   2.   3.
   3.   3.   3.   5.   3.   3.   3.   3.   3.   3.   3.   4.   4.   4.
   4. 

In [ ]:
bufTemp = 0
y = np.loadtxt('./train/train.txt', np.dtype('float64'))
# index = np.empty((frameCount-1), np.dtype('int'))

m = frameCount-1
trainM = int(m*.6)
valM = int((m - trainM)*.5)
testM = m-(trainM + valM)

rng_state = np.random.get_state()
np.random.shuffle(x)
np.random.set_state(rng_state)
np.random.shuffle(y)
# np.random.set_state(rng_state)
# np.random(index)

trainX = x[0:trainM]
valX = x[trainM:trainM+valM]
testX = x[trainM+valM:trainM+valM+testM]

trainY = y[1:trainM+1]
valY = y[trainM+1:trainM+valM+1]
testY = y[trainM+valM+1:trainM+valM+testM+1]
# y = 0
print(m, trainM, valM, testM)
print(len(trainX),len(trainY),len(valX),len(valY),len(testX),len(testY))

# Alpha greater than 0.16 will overflow and not work.
# Alpha around 0.01 or lower seems to do the best.
# alpha2 = np.array([0.00000610351563,0.0000122070313,0.0000244140625,0.000048828125,0.00009765625,0.0001953125,0.000390625,0.00078125,0.0015625,0.003125,0.00625,0.0125,0.025,0.05,0.1],np.dtype('float64'))
# alpha = np.array([0.000005],np.dtype('float64'))
# alpha = np.array([0.00006],np.dtype('float64'))
alpha = np.array([0.0001],np.dtype('float64'))
# lmbda = np.array([0.64],np.dtype('float64'))
lmbda = np.array([1.5],np.dtype('float64'))

# outerLoopCount = len(alpha2)
outerLoopCount = 15
loopCount = 120 # Was 5
costOnLoop = 2

difMul = 0
difDiv = 1


alpLmbTheta = np.zeros((len(alpha), len(lmbda), ((frameHeight*frameWidth)+1)), np.dtype('float64'))

alpLmbCost = np.zeros((len(alpha), len(lmbda)), np.dtype('float64'))
costArr = np.zeros((1),np.dtype('float64'))
beginArr = 1

result = np.empty((len(alpha), len(lmbda), frameCount), np.dtype('float64'))

stop = False
threshold = 5.0

count = 0

In [ ]:
print(x)

In [ ]:
# Uses bufTemp, y, m, alpha, lmbda, alpLmbTheta, loopCount, costArr
def learn():
    global costArr, beginArr, difDivider
    for idx, alp in enumerate(alpha):
        for idx2, lmbd in enumerate(lmbda):
            for i in range(loopCount):
                for j in range(trainM):
                    z = np.array(trainX[j], np.dtype('float64'))
                    grad = ((np.dot(z, alpLmbTheta[idx, idx2]) - trainY[j])*z)/m
                    grad[1:len(grad)] = grad[1:len(grad)] + ((lmbd/m)*alpLmbTheta[idx,idx2,1:len(alpLmbTheta[idx,idx2])])
                    alpLmbTheta[idx,idx2] = (alpLmbTheta[idx,idx2] - (alp*grad))
                if i % costOnLoop == 0:
                    cost = np.zeros((1), np.dtype('float64'))
                    for j in range(trainM):
                        z = np.array(trainX[j], np.dtype('float64'))
                        result[idx][idx2][j] = np.dot(z,alpLmbTheta[idx,idx2])
                        cost = cost + ((result[idx][idx2][j] - y[j])**2)
                    cost = cost/(trainM)
                    if len(costArr) < 500:
                        costArr = np.append(costArr, cost)
                    print("%d:"%(i), cost)
                    if i >= costOnLoop*2:
                        if (costArr[-2]-costArr[-1])<(((costArr[beginArr]-costArr[beginArr+1])*difMul)/difDiv) or costArr[-2]-costArr[-1] < 0:
                            alpha[idx] = alp/2
                            beginArr = len(costArr)
                            return 1
            print()
    return 0

In [ ]:
def startLearn():
    learn()
    while costArr[-1] > threshold:
        learn()

In [ ]:
def startLearnRange():
    for i in range(outerLoopCount):
        print(i, alpha[0], lmbda[0])
        if learn() == 0:
            alpha[0] = alpha[0]/2
#         print()

In [ ]:
def getResult(X, setM):
    
    for idx in range(len(alpha)):
        for idx2 in range(len(lmbda)):
            for j in range(setM):
                z = np.array(X[j], np.dtype('float64'))
                result[idx][idx2][j] = np.dot(z,alpLmbTheta[idx,idx2])

#     result.tofile('./train/trainMini.txt',sep="\n",format="%s")

In [ ]:
def getCost(Y, setM):
    cost = np.zeros((1), np.dtype('float64'))
    
    for idx in range(len(alpha)):
        for idx2 in range(len(lmbda)):
            for j in range(setM):
#                 cost = cost + (((result[idx][idx2][j] - y[j])**2)/2)
                cost = cost + ((result[idx][idx2][j] - Y[j])**2)
#             cost = cost/(2*m)
            cost = cost/(setM)
            print(lmbda[idx2], ":", cost)
#             alpLmbCost[idx,idx2] = cost
#             print(alpha[idx], ":", cost)
        print()

In [ ]:
startLearnRange()

In [ ]:
getResult(trainX, trainM)
print("Train:")
getCost(trainY, trainM)
getResult(valX, valM)
print("Val:")
getCost(valY, valM)
getResult(testX, testM)
print("Test:")
getCost(testY, testM)

In [ ]:
print(costArr)

In [ ]:
print(trainX[2])
print()
# print(alpLmbTheta)

In [ ]:
print(np.dot(trainX[0],alpLmbTheta[0,0]))
print(trainY[0])

In [ ]:
print(alpLmbTheta)

In [ ]:
index = np.zeros((1),np.dtype('int'))
for i in range(len(costArr)):
    index = np.append(index,i)

In [ ]:
plt.plot(index,costArr)

In [ ]:
getResult(trainX, trainM)
print("Train:")
getCost(trainY, trainM)
getResult(valX, valM)
print("Val:")
getCost(valY, valM)
getResult(testX, testM)
print("Test:")
getCost(testY, testM)

In [ ]:
lmbda[0] = 0.64
startLearnRange()
getResult(trainX, trainM)
print("Train:")
getCost(trainY, trainM)
getResult(valX, valM)
print("Val:")
getCost(valY, valM)
getResult(testX, testM)
print("Test:")
getCost(testY, testM)

In [ ]:
alpLmbTheta = np.zeros((len(alpha), len(lmbda), ((frameHeight*frameWidth)+1)), np.dtype('float64'))
lmbda[0] = 1.28
startLearnRange()
getResult(trainX, trainM)
print("Train:")
getCost(trainY, trainM)
getResult(valX, valM)
print("Val:")
getCost(valY, valM)
getResult(testX, testM)
print("Test:")
getCost(testY, testM)

In [ ]:
alpLmbTheta = np.zeros((len(alpha), len(lmbda), ((frameHeight*frameWidth)+1)), np.dtype('float64'))
lmbda[0] = 2.56
startLearnRange()
getResult(trainX, trainM)
print("Train:")
getCost(trainY, trainM)
getResult(valX, valM)
print("Val:")
getCost(valY, valM)
getResult(testX, testM)
print("Test:")
getCost(testY, testM)

In [ ]:
alpLmbTheta = np.zeros((len(alpha), len(lmbda), ((frameHeight*frameWidth)+1)), np.dtype('float64'))
lmbda[0] = 5.12
startLearnRange()
getResult(trainX, trainM)
print("Train:")
getCost(trainY, trainM)
getResult(valX, valM)
print("Val:")
getCost(valY, valM)
getResult(testX, testM)
print("Test:")
getCost(testY, testM)

In [ ]:
alpLmbTheta[0][0] = np.loadtxt('./train/trainThetaMini2.txt', np.dtype('float64'), delimiter='\n')
getResult()
getCost()

In [ ]:
alpLmbTheta.tofile('./train/trainThetaMini3.txt',sep="\n",format="%s")

In [ ]:
result.tofile('./train/trainMini.txt',sep="\n",format="%s")

In [ ]:
result = np.empty((len(alpha), len(lmbda), frameCount+2), np.dtype('float64'))
result[0][0] = np.loadtxt('./train/trainMini.txt', np.dtype('float64'), delimiter='\n')
getCost()

In [ ]:
# lmbda = 5.12 seems to be the best

In [ ]:
0 5e-06 1.5
0: [215.21518242]
2: [212.2981285]
4: [209.44135141]
6: [206.64360017]
8: [203.90364984]
10: [201.22030091]
12: [198.59237885]


In [ ]:
0 5e-06 1.5
0: [107.20166524]
2: [98.03785255]
4: [96.28437233]
6: [95.39492375]
1 2.5e-06 1.5
0: [95.1562898]
2: [94.73535018]
4: [94.38960088]
6: [94.09404927]
8: [93.83702913]

In [ ]:
14 6.103515625e-09 1.5
0: [87.187012]
2: [87.18703394]
4: [87.18705587]